<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Javier Santidrián Salas
- Nombre de alumno 2: Patricio Yáñez Alarcón


### **Link de repositorio de GitHub:** https://github.com/jsantidrian/labs-Santidrian-Yanez

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [30]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
#!pip install umap-learn

# Librería para NLP
#!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

# Librería para EDA
#!pip install pandas_profiling
from pandas_profiling import ProfileReport

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow.

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados.
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


Cargamos drive:

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargamos bases:

In [3]:
df_comics = pd.read_csv('/content/drive/My Drive/U/11 Semestre/Lab Progra/Labs/Lab 9/df_comics.csv')
df_comics_no_label = pd.read_csv('/content/drive/My Drive/U/11 Semestre/Lab Progra/Labs/Lab 9/comics_no_label.csv')
df_comics.dropna(subset=['history_text'],inplace=True) # eliminar ejemplos sin historia

Visualizamos con head los atributos de cada dataset:

In [4]:
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
df_comics_no_label.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,14,Ace Morgan,Kyle Morgan,NaN,7,NaN,"Aviation: Ace is an extremely skilled pilot, ...",85,20,65,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0


Notar que la columna "Unnamed: 0" no dice información útil, por lo tanto se eliminará de ambos dataframes:

In [6]:
df_comics.drop(columns=['Unnamed: 0'], inplace=True)
df_comics_no_label.drop(columns=['Unnamed: 0'], inplace=True)

Visualizamos la información completa de cada dataset:

In [7]:
df_comics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1285 entries, 0 to 1366
Data columns (total 81 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   name                              1285 non-null   object 
 1   real_name                         1164 non-null   object 
 2   full_name                         895 non-null    object 
 3   overall_score                     1285 non-null   object 
 4   history_text                      1285 non-null   object 
 5   powers_text                       961 non-null    object 
 6   intelligence_score                1285 non-null   int64  
 7   strength_score                    1285 non-null   int64  
 8   speed_score                       1285 non-null   int64  
 9   durability_score                  1285 non-null   int64  
 10  power_score                       1285 non-null   int64  
 11  combat_score                      1285 non-null   int64  
 12  superp

In [8]:
df_comics_no_label.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 81 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   name                              82 non-null     object 
 1   real_name                         66 non-null     object 
 2   full_name                         13 non-null     object 
 3   overall_score                     84 non-null     object 
 4   history_text                      76 non-null     object 
 5   powers_text                       43 non-null     object 
 6   intelligence_score                84 non-null     int64  
 7   strength_score                    84 non-null     int64  
 8   speed_score                       84 non-null     int64  
 9   durability_score                  84 non-null     int64  
 10  power_score                       84 non-null     int64  
 11  combat_score                      84 non-null     int64  
 12  superpower

EDA:

In [9]:
#profile_comics = ProfileReport(df_comics, title='EDA Report - Comics Dataset')
#profile_comics_no_label = ProfileReport(df_comics_no_label, title='EDA Report - Comics Dataset (No Label)')

#profile_comics.to_file(output_file='eda_comics.html')
#profile_comics_no_label.to_file(output_file='eda_comics_no_label.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset.

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen.

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?.

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [10]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [11]:
stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [12]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [13]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados.

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [14]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

Generamos objetos CountVectorizer() y MinMaxScaler() solicitados:

In [15]:
bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )

scaler = MinMaxScaler()

Seleccionamos atributos de interés:

In [16]:
atributos_de_interes = ['intelligence_score',
                        'strength_score',
                        'speed_score',
                        'durability_score',
                        'power_score',
                        'combat_score']

Hacemos un ColumnTransformer():

In [17]:
codificacion = ColumnTransformer(
    transformers=[
        ('Caracteristicas', bog, 'history_text'),
        ('Escalamiento', scaler, atributos_de_interes)
    ]
)

Generamos un pipeline con las transformaciones:

In [18]:
pipeline_cod = Pipeline([
    ('codificacion', codificacion)
])

Aplicamos el pipeline de transformaciones:



In [19]:
pipeline_cod.fit_transform(df_comics)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<1285x307713 sparse matrix of type '<class 'numpy.float64'>'
	with 1068237 stored elements in Compressed Sparse Row format>

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`.

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

Obtenemos los valores de los labels de la variable objetivo y separamos el resto de features (dropearemos las columnas name, real name, full name puesto que no aportan para la predicción posterior ya que tienen una función meramente de id):

In [20]:
y = df_comics['alignment']
X = df_comics.drop(columns=['alignment', 'name', 'real_name', 'full_name'])

Conteo de datos por clase:

In [21]:
y.value_counts()

Good       743
Bad        429
Neutral    113
Name: alignment, dtype: int64

Encodeamos las features de acuerdo a observaciones:

- La variable tipo texto history_text será procesada mediante bag of words como fue hecho en la sección 1.1
- Todas las variables que tienen prefijo has_ ['has_electrokinesis',
 'has_energy_constructs',
 'has_mind_control_resistance',
 'has_matter_manipulation',
 'has_telepathy_resistance',
 'has_mind_control',
 'has_enhanced_hearing',
 'has_dimensional_travel',
 'has_element_control',
 'has_size_changing',
 'has_fire_resistance',
 'has_fire_control',
 'has_dexterity',
 'has_reality_warping',
 'has_illusions',
 'has_energy_beams',
 'has_peak_human_condition',
 'has_shapeshifting',
 'has_heat_resistance',
 'has_jump',
 'has_self-sustenance',
 'has_energy_absorption',
 'has_cold_resistance',
 'has_magic',
 'has_telekinesis',
 'has_toxin_and_disease_resistance',
 'has_telepathy',
 'has_regeneration',
 'has_immortality',
 'has_teleportation',
 'has_force_fields',
 'has_energy_manipulation',
 'has_endurance',
 'has_longevity',
 'has_weapon-based_powers',
 'has_energy_blasts',
 'has_enhanced_senses',
 'has_invulnerability',
 'has_stealth',
 'has_marksmanship',
 'has_flight',
 'has_accelerated_healing',
 'has_weapons_master',
 'has_intelligence',
 'has_reflexes',
 'has_super_speed',
 'has_durability',
 'has_stamina',
 'has_agility',
 'has_super_strength'] son binarias (valores 0 o 1, de tipo float64), y como tienen valores nulos (muy pocos en comparación al total), aquellos valores serán imputados por la el valor mayoritario.
- Las variables ['powers_text', 'superpowers', 'alter_egos', 'aliases', 'place_of_birth', 'first_appearance', 'creator', 'occupation', 'base', 'teams', 'relatives', 'gender', 'type_race', 'height', 'weight', 'eye_color', 'hair_color', 'skin_color', 'img'] serán procesadas por one hot encoding dada su naturaleza categórica nominal.
- La variable overall_score tiene valores enteros pero en formato string, dentro de los cuales hay también valores '$\infty$' y '-', por lo tanto lo primero que haremos será imputar aquellos strings por el string (valor entero) mayoritario. Luego transformaremos estos strings con valores numéricos a formato numérico (int64) y aplicaremos MinMaxScaler a las variables numéricas enteras ['overall_score', 'intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

Comenzamos imputando los valores nulos de las variables binarias (valores 0 o 1) que contienen has_ por el valor mayoritario:

In [22]:
variables_has = ['has_electrokinesis',
                'has_energy_constructs',
                'has_mind_control_resistance',
                'has_matter_manipulation',
                'has_telepathy_resistance',
                'has_mind_control',
                'has_enhanced_hearing',
                'has_dimensional_travel',
                'has_element_control',
                'has_size_changing',
                'has_fire_resistance',
                'has_fire_control',
                'has_dexterity',
                'has_reality_warping',
                'has_illusions',
                'has_energy_beams',
                'has_peak_human_condition',
                'has_shapeshifting',
                'has_heat_resistance',
                'has_jump',
                'has_self-sustenance',
                'has_energy_absorption',
                'has_cold_resistance',
                'has_magic',
                'has_telekinesis',
                'has_toxin_and_disease_resistance',
                'has_telepathy',
                'has_regeneration',
                'has_immortality',
                'has_teleportation',
                'has_force_fields',
                'has_energy_manipulation',
                'has_endurance',
                'has_longevity',
                'has_weapon-based_powers',
                'has_energy_blasts',
                'has_enhanced_senses',
                'has_invulnerability',
                'has_stealth',
                'has_marksmanship',
                'has_flight',
                'has_accelerated_healing',
                'has_weapons_master',
                'has_intelligence',
                'has_reflexes',
                'has_super_speed',
                'has_durability',
                'has_stamina',
                'has_agility',
                'has_super_strength']

for col in variables_has:
  moda = X[col].mode()[0]
  X[col].fillna(moda, inplace=True)

Creamos la lista de variables categóricas nominales a procesar con one hot encoding:

In [23]:
nominales = ['powers_text',
             'superpowers',
             'alter_egos',
             'aliases',
             'place_of_birth',
             'first_appearance',
             'creator',
             'occupation',
             'base',
             'teams',
             'relatives',
             'gender',
             'type_race',
             'height',
             'weight',
             'eye_color',
             'hair_color',
             'skin_color',
             'img']

Imputamos los 2 valores diferentes ('$\infty$' y '-') en variable overall_score por el valor mayoritario, luego convertimos a numérico (int64) y creamos la lista de variables numéricas enteras:

In [24]:
moda = X['overall_score'].mode()[0]
X['overall_score'].replace({'-': moda, '∞': moda}, inplace=True)
X['overall_score'] = X['overall_score'].astype(int)

numericas = ['overall_score',
             'intelligence_score',
             'strength_score',
             'speed_score',
             'durability_score',
             'power_score',
             'combat_score']

Dividimos en train-test en proporción 80-20 de manera estratificada (es decir, manteniendo la distribución de las clases):

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Procedemos a generar el pipeline de clasificación con multinomial naive bayes buscado:

In [31]:
# Bag of Words para variable 'history text'
bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )

# Escalador para variables numéricas enteras
scaler = MinMaxScaler()

# One Hot Encoder para variables categóricas nominales
onehot = OneHotEncoder(sparse=False,handle_unknown='ignore')

# Selector de features
selector = SelectPercentile(score_func=f_classif, percentile=90)

# Modelo de clasificación multinomial naive bayes
multinomial_nb = MultinomialNB()

# Column Transformer de preprocesamiento
preprocesamiento = ColumnTransformer(
    transformers=[
        ('Caracteristicas', bog, 'history_text'),
        ('Escalamiento', scaler, numericas),
        ('Codificacion', onehot, nominales)
    ]
)

# Pipeline de clasificación con multinomial naive bayes
pipeline_nb = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('seleccion', selector),
    ('clasificacion', multinomial_nb)
])

# Entrenamos el clasificador
pipeline_nb.fit(X_train, y_train)

# Calculamos predicciones
y_pred_nb = pipeline_nb.predict(X_test)

# Imprimimos métricas de evaluación
print(classification_report(y_test, y_pred_nb))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


              precision    recall  f1-score   support

         Bad       0.70      0.16      0.26        86
        Good       0.61      0.98      0.75       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.62       257
   macro avg       0.44      0.38      0.34       257
weighted avg       0.59      0.62      0.52       257



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


- El valor de accuracy es bastante bajo (0.62), lo cual implica que la clasificación no es muy buena (es decir, no se puede predecir con mucha exactitud cual es la alineación de un personaje de un cómic).
- Notamos que hay un claro desbalance de clases, lo cual se refleja en las métricas de preicision, recall y f1-score obtenidas para cada clase (notar que en particular hay un grave problema en la clase neutral, pues tiene las 3 métricas iguales a 0). Si observamos el valor de las métricas promediadas ponderadas (weighted avg) se observa que tenemos valores bajos para precision, recall y f1-score, por lo que este clasificador no es muy bueno, ya que hay mucha cantidad de falsos positivos y negativos.

Ahora hacemos lo mismo pero con DummyClassifier:

In [32]:
# Bag of Words para variable 'history text'
bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )

# Escalador para variables numéricas enteras
scaler = MinMaxScaler()

# One Hot Encoder para variables categóricas nominales
onehot = OneHotEncoder(sparse=False,handle_unknown='ignore')

# Selector de features
selector = SelectPercentile(score_func=f_classif, percentile=90)

# Modelo de clasificación multinomial naive bayes
dummy = DummyClassifier()

# Column Transformer de preprocesamiento
preprocesamiento = ColumnTransformer(
    transformers=[
        ('Caracteristicas', bog, 'history_text'),
        ('Escalamiento', scaler, numericas),
        ('Codificacion', onehot, nominales)
    ]
)

# Pipeline de clasificación con dummyclassifier
pipeline_dc = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('seleccion', selector),
    ('clasificacion', dummy)
])

# Entrenamos el pipeline de clasificacion
pipeline_dc.fit(X_train, y_train)

# Calculamos predicciones de la clasificación
y_pred_dc = pipeline_dc.predict(X_test)

# Imprimimos métricas de evaluación
print(classification_report(y_test, y_pred_dc))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


              precision    recall  f1-score   support

         Bad       0.00      0.00      0.00        86
        Good       0.58      1.00      0.73       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.58       257
   macro avg       0.19      0.33      0.24       257
weighted avg       0.33      0.58      0.42       257



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


- El valor de accuracy obtenido es de 0.58, que es muy cercano al obtenido antes con multinomial naive bayes (0.62). Esto implica que el modelo de naive bayes multinomial es peor de lo que pensabamos (pues predice practicamente con la misma exactitud que un modelo que predice aleatoriamente como dummy classifier).
- Los valores de las 3 métricas precision, recall y f1-score son casi iguales de bajos que los obtenidos anteriormente, lo que prueba nuevamente que el modelo anterior no predice para nada bien, pues tiene problemas con falsos positivos y negativos.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

Usamos pipeline base (primer paso: preprocesamiento, segundo paso: selección de mejores features, tercer paso: clasificador):

In [40]:
# Bag of Words para variable 'history text'
bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2)
                      )

# Escalador para variables numéricas enteras
scaler = MinMaxScaler()

# One Hot Encoder para variables categóricas nominales
onehot = OneHotEncoder(sparse=False,handle_unknown='ignore')

# Selector de features
selector = SelectPercentile(score_func=f_classif, percentile=90)

# Modelo de clasificación SVC
modelo = SVC(random_state=42)

# Column Transformer de preprocesamiento
preprocesamiento = ColumnTransformer(
    transformers=[
        ('caracteristicas', bog, 'history_text'),
        ('escalamiento', scaler, numericas),
        ('codificacion', onehot, nominales)
    ]
)

# Pipeline de clasificación con modelo SVC
pipeline = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('seleccion', selector),
    ('clasificacion', modelo)
])

Creamos la grilla (3 posibilidades de clasificadores, 4 posibilidades para percentil en SelectPercentile y 3 posibilidades para ngram_range en CountVectorizer):

In [41]:
params = [
    {
        "clasificacion": [SVC(random_state=42),
                          LogisticRegression(random_state=42),
                          RandomForestClassifier(random_state=42)],
        "seleccion__percentile": [20,
                                  40,
                                  60,
                                  80],
        "preprocesamiento__caracteristicas__ngram_range": [(1, 1),
                                                           (1, 2),
                                                           (1, 3)]
    }
]

Realizamos la búsqueda de parámetros:

In [42]:
gs = HalvingGridSearchCV(pipeline, params, n_jobs=-1, scoring="f1_macro", verbose=10).fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 38
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 36
n_resources: 38
Fitting 5 folds for each of 36 candidates, totalling 180 fits
----------
iter: 1
n_candidates: 12
n_resources: 114
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 2
n_candidates: 4
n_resources: 342
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 3
n_candidates: 2
n_resources: 1026
Fitting 5 folds for each of 2 candidates, totalling 10 fits


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Mejor score obtenido para el gridsearch:

In [43]:
gs.best_score_

0.3449861101360523

Mejor estimador obtenido mediante el gridsearch:

In [44]:
gs.best_estimator_

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('caracteristicas',
                                                  CountVectorizer(ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f097ec98190>),
                                                  'history_text'),
                                                 ('escalamiento',
                                                  MinMaxScaler(),
                                                  ['overall_score',
                                                   'intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score']),
                                                 ('...
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['powers_text', 'superpowers',
                                                   'alter_egos', 'aliases',
                                                   'place_of_birth',
                                                   'first_appearance',
                                                   'creator', 'occupation',
                                                   'base', 'teams', 'relatives',
                                                   'gender', 'type_race',
                                                   'height', 'weight',
                                                   'eye_color', 'hair_color',
                                                   'skin_color', 'img'])])),
                ('seleccion', SelectPercentile(percentile=20)),
                ('clasificacion', RandomForestClassifier(random_state=42))])

Mediante el gridsearch se obtuvo que el mejor estimador corresponde a:

- Clasificador: Random Forest (este en general sirve mejor para entrenamientos y predicciones de datos con muchas variables).
- Percentil para la selección de mejores features: 20% (lo cual puede ser debido a que en general son pocas las features que definen una predicción).
- Rango del n-grama: (1,2) (es más útil trabajar bigramas en general debido a la estructura de texto).



## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

Redefinimos X_test ahora con la data no etiquetada df_comics_no_label:

In [49]:
X_test = df_comics_no_label.drop(columns=['alignment', 'name', 'real_name', 'full_name'])

Creamos el pipeline óptimo de acuerdo al gridsearch:

In [54]:
# Bag of Words para variable 'history text' con bigramas óptimo
bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2)
                      )

# Escalador para variables numéricas enteras
scaler = MinMaxScaler()

# One Hot Encoder para variables categóricas nominales
onehot = OneHotEncoder(sparse=False,handle_unknown='ignore')

# Selector de features percentil 20 óptimo
selector = SelectPercentile(score_func=f_classif, percentile=20)

# Modelo de clasificación Random Forest óptimo
modelo = RandomForestClassifier(random_state=42)

# Column Transformer de preprocesamiento
preprocesamiento = ColumnTransformer(
    transformers=[
        ('caracteristicas', bog, 'history_text'),
        ('escalamiento', scaler, numericas),
        ('codificacion', onehot, nominales)
    ]
)

# Pipeline de clasificación con modelo SVC
pipeline = Pipeline([
    ('preprocesamiento', preprocesamiento),
    ('seleccion', selector),
    ('clasificacion', modelo)
])

# Entrenamos el pipeline de clasificacion
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('caracteristicas',
                                                  CountVectorizer(ngram_range=(1,
                                                                               2),
                                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f097f2a2bf0>),
                                                  'history_text'),
                                                 ('escalamiento',
                                                  MinMaxScaler(),
                                                  ['overall_score',
                                                   'intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score']),
                                                 ('...
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['powers_text', 'superpowers',
                                                   'alter_egos', 'aliases',
                                                   'place_of_birth',
                                                   'first_appearance',
                                                   'creator', 'occupation',
                                                   'base', 'teams', 'relatives',
                                                   'gender', 'type_race',
                                                   'height', 'weight',
                                                   'eye_color', 'hair_color',
                                                   'skin_color', 'img'])])),
                ('seleccion', SelectPercentile(percentile=20)),
                ('clasificacion', RandomForestClassifier(random_state=42))])

Ahora hay que realizar las predicciones:

In [ ]:
# Calculamos predicciones de la clasificación
y_pred = pipeline.predict(X_test)

Falta correr eso último para conocer las predicciones de Vergil, Gorilla Girl y Batcow

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>